In [1]:
from pyomo.environ import *
import sys
from itertools import product

sys.path.append('../../../src')
# from energiapy.utils.scale_utils import scale_list, scale_tuple

In [2]:
T_up = 10
T = [i for i in range(0,T_up)]
# structure = [4, 7]
# time_index_dict = dict()


In [3]:
m = ConcreteModel()

In [4]:
m.time = Set(initialize=T)
m.sources = Set(initialize=['A'])
m.sinks = Set(initialize=['B'])
m.transports = Set(initialize=['truckAB', 'planeAB'])
m.resources = Set(initialize=['resource1'])

In [5]:
m.Exp = Var(m.sources, m.sinks, m.transports, m.resources, m.time, domain=NonNegativeReals)
m.Transit = Var(m.sources, m.sinks, m.transports, m.resources, m.time, domain=NonNegativeReals)

In [6]:
travel_time_dict = {('A', 'B'): {'truckAB': 5, 'planeAB': 2}}

transit_initial_dict = {('A', 'B', 'truckAB', 'resource1', 1): 45,
                        ('A', 'B', 'planeAB', 'resource1', 3): 65}

In [7]:
list(transit_initial_dict.keys())

[('A', 'B', 'truckAB', 'resource1', 1), ('A', 'B', 'planeAB', 'resource1', 3)]

In [8]:
# m.Exp.pprint()

In [9]:
scales = [tuple(T)]
scale_iter = list(product(*scales))

In [10]:
def constraint_transit_balance(instance: ConcreteModel) -> Constraint:
    
    def transit_balance_rule(instance: ConcreteModel, source, sink, transport, resource, *scale_list):
            
        current_index = scale_iter.index(scale_list[:0 + 1])
        difference = scale_iter.index(scale_list[:0 + 1]) - travel_time_dict[(source, sink)][transport]
        
        if current_index - 1 >= 0:
            previous = instance.Transit[source, sink, transport, resource, scale_iter[current_index - 1]]
        else:
            previous = 0
            
        incoming = instance.Exp[source, sink, transport, resource, scale_iter[current_index]]
        
        if difference >= 0:
            outgoing = instance.Exp[source, sink, transport, resource, scale_iter[difference]]
        else:
            outgoing = 0
            
        if (source, sink, transport, resource, *scale_list) in list(transit_initial_dict.keys()):
            initial = transit_initial_dict[source, sink, transport, resource, *scale_list]
        else:
            initial = 0
        
        return instance.Transit[source, sink, transport, resource, scale_iter[current_index]] == previous + incoming - outgoing + initial
    
    
    instance.constraint_transit_balance = Constraint(instance.sources, instance.sinks, instance.transports, instance.resources, *scales, rule=transit_balance_rule)
        

In [11]:
constraint_transit_balance(m)

In [12]:
m.constraint_transit_balance.pprint()

constraint_transit_balance : Size=20, Index=sources*sinks*transports*resources*{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}, Active=True
    Key                                   : Lower : Body                                                                                                                                     : Upper : Active
    ('A', 'B', 'planeAB', 'resource1', 0) :   0.0 :                                                                          Transit[A,B,planeAB,resource1,0] - Exp[A,B,planeAB,resource1,0] :   0.0 :   True
    ('A', 'B', 'planeAB', 'resource1', 1) :   0.0 :                                     Transit[A,B,planeAB,resource1,1] - (Transit[A,B,planeAB,resource1,0] + Exp[A,B,planeAB,resource1,1]) :   0.0 :   True
    ('A', 'B', 'planeAB', 'resource1', 2) :   0.0 :      Transit[A,B,planeAB,resource1,2] - (Transit[A,B,planeAB,resource1,1] + Exp[A,B,planeAB,resource1,2] - Exp[A,B,planeAB,resource1,0]) :   0.0 :   True
    ('A', 'B', 'planeAB', 'resource1', 3) :   0.0 : T

In [13]:
m.Transit.pprint()

Transit : Size=20, Index=sources*sinks*transports*resources*time
    Key                                   : Lower : Value : Upper : Fixed : Stale : Domain
    ('A', 'B', 'planeAB', 'resource1', 0) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 1) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 2) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 3) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 4) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 5) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 6) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 7) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1

In [14]:
m.constraint_transit_balance['A','B', 'planeAB', 'resource1', 3].pprint()

{Member of constraint_transit_balance} : Size=20, Index=sources*sinks*transports*resources*{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}, Active=True
    Key                                   : Lower : Body                                                                                                                                     : Upper : Active
    ('A', 'B', 'planeAB', 'resource1', 3) :   0.0 : Transit[A,B,planeAB,resource1,3] - (Transit[A,B,planeAB,resource1,2] + Exp[A,B,planeAB,resource1,3] - Exp[A,B,planeAB,resource1,1] + 65) :   0.0 :   True


In [15]:
m.Transit[('A','B','planeAB', 'resource1',0)].value = 25
m.Transit[('A','B','planeAB', 'resource1',0)].fixed = True

m.Transit[('A','B','planeAB', 'resource1',1)].value = 50
m.Transit[('A','B','planeAB', 'resource1',1)].fixed = True

In [16]:
m.Transit.pprint()

Transit : Size=20, Index=sources*sinks*transports*resources*time
    Key                                   : Lower : Value : Upper : Fixed : Stale : Domain
    ('A', 'B', 'planeAB', 'resource1', 0) :     0 :    25 :  None :  True : False : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 1) :     0 :    50 :  None :  True : False : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 2) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 3) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 4) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 5) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 6) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 7) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1

In [17]:
m2 = ConcreteModel()

In [18]:
m2.Exp = Var(m.sources, m.sinks, m.transports, m.resources, m.time, domain=NonNegativeReals)
m2.Transit = Var(m.sources, m.sinks, m.transports, m.resources, m.time, domain=NonNegativeReals)

In [21]:
def fix_variables(model1: ConcreteModel, model2: ConcreteModel, scen_name: str = None, current_time_idx: int = 0):
    model2_vars = {v.name: v for v in model2.component_objects(Var)}
    
    for var1 in model1.component_objects(Var):
        if var1.name in model2_vars:
            var2 = model2_vars[var1.name]
            for key in (k for k in var1.keys() if k[-1] < current_time_idx):
                var2[key].fixed = True
                var2[key] = var1[key].value


In [22]:
fix_variables(model1=m, model2=m2, current_time_idx=1)

In [23]:
m2.Transit.pprint()

Transit : Size=20, Index=sources*sinks*transports*resources*time
    Key                                   : Lower : Value : Upper : Fixed : Stale : Domain
    ('A', 'B', 'planeAB', 'resource1', 0) :     0 :    25 :  None :  True : False : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 1) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 2) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 3) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 4) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 5) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 6) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 7) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1

In [24]:
m2.Exp.pprint()

Exp : Size=20, Index=sources*sinks*transports*resources*time
    Key                                   : Lower : Value : Upper : Fixed : Stale : Domain
    ('A', 'B', 'planeAB', 'resource1', 0) :     0 :  None :  None :  True :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 1) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 2) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 3) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 4) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 5) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 6) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 7) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 8